In [99]:
class Value:
    
    def __init__(self, value, inputs = (), op = "", label = ""):
        self.value = value
        self.predecesors = set(inputs)
        self.op = op
        self.grad = 0
        self.label = label
        self.backward = lambda: None
        
    def __repr__(self) -> str:
        return (f"value of {self.label} = {self.value}; gradient = {self.grad}")
    
    def __add__(self, other):
        
        # calculate the gradient for addition
        def backward():
            self.grad = out.grad
            other.grad = out.grad
        
        # calculate the output of operation and assign it's local gradient 
        out =  Value(value = self.value + other.value, inputs=(self, other), op = "+")
        out.backward = backward
        
        #return the value
        return out
    
    def __mul__(self, other):
        
        # calculate the _gradient for multiplication
        def backward():
            self.grad = out.grad * other.value
            other.grad = out.grad * self.value
        
        # calculate the outpout of the operation and assign it it's respective backward function
        out = Value(value = self.value * other.value, inputs=(self, other), op = "*")
        out.backward = backward
        
        # return the output
        return out
    
    def backPropagation(self):
        '''
        Start the backpropagation from this node
        '''
        
        # set the gradient of this node to 1
        self.grad = 1
        
        # build a helper function to create the topological sorting of the graph
        def topologicalSorting(node):
            # create local variable
            L = []
            s = set()
            
            # recursive sorting
            def topo(node:Value):
                if node not in s:
                    s.add(node)
                    for parent in node.predecesors:
                        topo(parent)
                    L.append(node)
            topo(node)
            return L
        
        # call the backward method on each node, starting from the last one
        for node in topologicalSorting(self)[::-1]:
            node.backward()
        

In [103]:
a = Value(2, label="a")
b = Value(-3, label="b")
c = a + b; c.label="c"
d = c * b; d.label = "d"

d.backPropagation()

c.grad

-3